<a href="https://colab.research.google.com/github/Eran707/MSc-Computational-Neuroscience-Repo/blob/master/V4_Single_Compartment_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Single compartment pump leak model

Coded by Eran Shorer (piggybacking on Kira Dusterwald and Alan Kay)

## Introduction

Herein I attempt to model the dynamics of a single neuron with emphasis Chloride homeostasis to better understand processes occuring in epilepsy. 

A Neuron is a type of cell in the nervous system that is able to communicate with other neurons via electrochemical signals. 
The trillions of communications that occur in the brain give rise to complex animal behaviours.
Errors in this signalling pathway therefore can lead disorders in the nervous system.
To understand how a network of cells function we can start off by understanding the dynamics of a single neuron. 





![alt text](https://drive.google.com/uc?id=1GT7WU9e0qYCdLHlDstU_SonNyBc5PP3b)

Above we have the cell body of a single neuron. 

(A) Ions:


*   Sodium (Na), charge of +1
*   Potassium (K), charge of +1
*   Chloride (Cl), charge of -1
*   Impermeant anions (X) charge of z

(B) ATPase: 

ATP dependant Sodium Potassium Pump which pumps 3 Na out of the neuron for every 2 K it pumps in.

(C) KCC2:

Potassium-Chloride Cotransporter 2. Pumps out potassium and chloride.



## Module installation:

numpy - for mathematical operations
matplotlib - for graphing
seaborn - for graphing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
print('modules installed')

ERROR! Session/line number was not unique in database. History logging moved to new session 59
modules installed


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Constants:

R = Gas Constant
F = Faradays Constant
T = Temperature in Kelvin
RTF is used as a combined variable


In [2]:
R = 8.31466  # J/(Kelvin.mol) #Universal Gas Constant
F = 96485.0 #C/mol # Faraday Constant in Volts
T = 310.15   #Kelvin # Absolute temperature (37C)
RTF = R*T/F  # J/C
pw = 0.0015 #dm/s #osmotic permeability of biological membranes
vw = 0.018 #dm^3/mol #partial molar volume of water
print('Constants set')

Constants set


## Parameters:

### Cellular dimensions
We will be using a cylindrical model of the cell. 
The area of the sphere (SA) will be fixed but the volume (w) will change based on the osmotic properties of the cell.
C_m refers to the membrane capacitance and the FinvCSA is a composite parameter.


![Single compartment dimensions](https://drive.google.com/open?id=1ewq1gO9pru_DFCX4OXwM38dYA_UuCwfe)

In [3]:
rad = 5e-5                      #Radius of compartment (dm)
length = 25e-5                  #Length of compartment (dm) 

w_initial = np.pi*rad**2*length #Initial volume in litres
w = w_initial
SA_initial = 2*np.pi*rad*length #Initial surface area in dm^2
SA = SA_initial 
Ar = w/SA               #Area scaling constant dm^-1

C_m = 2e-4              #(F/dm^2) #Unit of membrane capacitance
FinvCAr = F/(C_m)*Ar
print('Initial compartment volume:', w_initial, 'Litres')

Initial compartment volume: 1.9634954084936206e-12 Litres


Q: In Kira's code there is F1D optimization. What is this?


### Ionic properties 

Here I will define the baseline Osmolarity and hence calculate the concentrations of the ions of interest (Na+, K+, Cl-, and X(impermeant anions)). Both the extracellular and intracellular osmolarities are calculated here.
We start by setting the osmolarity to be 300 x 10^-3 Molar. 
Omsols of negative and positive ions must be 150 X10^-3 Molar respectively. 

In [4]:
#Extracellular concentrations (M)
ConcO_K = 3.5e-3             
ConcO_Cl= 119e-3      
ConcO_Na = 145e-3 
ConcO_X = -1*(ConcO_Cl - ConcO_Na - ConcO_K)*0.2
z_X = -0.85 

OsmolO = ConcO_K + ConcO_Na + ConcO_Cl + ConcO_X

#Intracellular concentrations (M)
ConcI_K = 122.873e-3
ConcI_Cl = 5.163e-3
ConcI_Na = 14.002e-3
ConcI_X = 154.962e-3

OsmolI = ConcI_K + ConcI_Na + ConcI_Cl + ConcI_X

ratio_X = 0.98 # set ratio of intracellular:extracellular impermeants

print("Extracellular concentrations: (Molar)")
print("---------------------")
print('Na: ',ConcO_Na)
print('K: ',ConcO_K)
print('Cl: ', ConcO_Cl)
print('X: ', ConcO_X)
print('Extracellular Osmolarity:', OsmolO)
print("")
print("Intracellular concentrations: (Molar)")
print("---------------------")
print('Na: ',ConcI_Na)
print('K: ',ConcI_K)
print('Cl: ', ConcI_Cl)
print('X: ', ConcI_X)
print('Intracellular Osmolarity:', OsmolI)
print("")

Extracellular concentrations: (Molar)
---------------------
Na:  0.145
K:  0.0035
Cl:  0.119
X:  0.005899999999999999
Extracellular Osmolarity: 0.2734

Intracellular concentrations: (Molar)
---------------------
Na:  0.014002
K:  0.122873
Cl:  0.005163
X:  0.154962
Intracellular Osmolarity: 0.297



Q: Where did the equation to calculate extracelluar X come from?

### Channel and pump properties

In [5]:
g_Na = 2e-3/F   # S/dm2 #Na Leak conductance  (All from Kira's code)
g_K = 7e-3/F    # S/dm2 #K Leak conductance
g_Cl =2e-3/F    # S/dm2 #Cl Leak conductance
g_KCC2 = 2e-3/F #KCC2 conductance from Kira's code
g_X = 0/F       # No conductance for impermeant anions

curr = -0*5e-8; #Baseline no current injected

p_default = -1
p_effective = (10**p_default)/F

print('Pump rate:', p_effective)

Pump rate: 1.0364305332435093e-06


Q: Unsure of what the p_default and p_effective are and how these calculations work

## Variable initialization:


### Timing related variables

In [6]:
#### - Timing related variables
t = 0               # real time in seconds
totalt = 200        # total time
dt = 1e-3           # time step
t_on = 0            # time when the ATPase is switched on
t_off = 5000        # time when the ATPase is switched off
totalsteps = round(totalt/dt)  # total number of time steps

sw = 1              # 0 = switch off; 1 = switch on
ctr = 1     # counter for plotting 
n = 1800     # number of plot points
ts = totalt/n   # time interval for plotting

print('Simulation will run for ',totalt,' seconds') 
print('The timestep will be', dt, 'seconds')
print('There will be ', totalsteps, ' time steps')
print('The ATPase pump is switched on at', t_on, 'seconds')
print('The ATPase pump is switched off at', t_off, 'seconds')

Simulation will run for  200  seconds
The timestep will be 0.001 seconds
There will be  200000  time steps
The ATPase pump is switched on at 0 seconds
The ATPase pump is switched off at 5000 seconds


### Array initialization

In [7]:
Na_Arr = []
K_Arr = []
Cl_Arr = []
X_Arr = []
Vm_Arr = []
t_Arr = []           
w_Arr = []
ECl_Arr =[]
EK_Arr = []

print('All arrays are empty to begin with')

All arrays are empty to begin with


# Simulation 

In order to calculate how the system changes with time I use a use a Euler approach with the following differential equaitons

**Calculating the membrane voltage change:**

\begin{equation}
\frac{\delta V_{m}}{\delta t}= \frac{F}{C_{m} SA} \times w \times ([Na] + [K] - [Cl] + z[X]) \\
\end{equation}

**Calculating the KCC2 Pump rate**

\begin{align}
E_{K} &= RTF \times log(\frac{[K]_{o}}{[K]_{i}})\\
E_{Cl} &= RTF \times log(\frac{[Cl]_{o}}{[Cl]_{i}})\\
J_{KCC2} &= g_{KCC2}\times(E_{K}-E_{Cl})\\
\nonumber
\end{align}

**Calculating changes to intracellular ion concentrations**
\begin{align}
\frac{\delta [Na]}{\delta t}&= -\dfrac{SA}{w}\times(g_{Na} \times(V_{m} - \dfrac{RT}{F} log(\dfrac{[Na]_{o}}{[Na]_{i}}) + 3J_{p}) \\
\frac{\delta [K]}{\delta t}&= -\dfrac{SA}{w} \times (g_{K} \times (V_{m} - \dfrac{RT}{F} log(\dfrac{[K]_{o}}{[K]_{i}}) + 2J_{p} - J_{KCC2}) \\
\frac{\delta [Cl]}{\delta t}&= \dfrac{SA}{w} \times  (g_{Cl} \times(V_{m} - \dfrac{RT}{F} log(\dfrac{[Cl]_{o}}{[Cl]_{i}}) - J_{KCC2}) \\
\nonumber
\end{align}




### Starting voltage:

In [8]:
Net_Intracellular_Charge = (ConcI_Na+ConcI_K)-ConcI_Cl+z_X*ConcI_X
Vm = FinvCAr*(Net_Intracellular_Charge)*1e3

print('Net Intracellular Charge:', Net_Intracellular_Charge)
print('F*w/Cm*SA :', FinvCAr)
print('Starting voltage of: ', round(Vm,3), 'mV')


Net Intracellular Charge: -5.699999999997374e-06
F*w/Cm*SA : 12060.624999999998
Starting voltage of:  -68.746 mV


### Starting ATPase Pump rate:

In [9]:
p_default = -1
p_effective = (10**p_default)/F

print('Pump rate:', p_effective)

Pump rate: 1.0364305332435093e-06


### Looping through time

In [9]:
for i in range(1,totalsteps):  #note tit = total number of timesteps 
    
    
    #Determining switch position of ATPase
    if (t<t_off) & (t>t_on): 
        sw=1 
    else: 
        sw=0
            
    # Voltage Calculation 
    Net_Intracellular_Charge = (ConcI_Na+ConcI_K)-ConcI_Cl+z_X*ConcI_X
    Vm = FinvCAr*(Net_Intracellular_Charge)
    
    # KCC2 pump rate calculation 
    EK = RTF*np.log(ConcO_K/ConcI_K)
    ECl = -RTF*np.log(ConcO_Cl/ConcI_Cl)
    JKCC2 = g_KCC2*(EK-ECl)

    # ATPase pump rate calclation
    Jp = p*(ConcI_Na/ConcO_Na)**3

    #Dummy X(impermeant anion) concentrations:
    ConcO_X_temp = ConcI_X*(1-X_ratio)    

    #Incrementing Ion concentration    
    d_Na = -dt/Ar*(g_Na*(Vm-RTF*np.log(ConcO_Na/ConcI_Na)) + sw*3*Jp) 
    d_K = -dt/Ar*(g_K*(Vm-RTF*np.log(ConcO_K/ConcI_K))-JKCC2-sw*2*Jp) 
    d_Cl = dt/Ar*(g_Cl*(Vm+RTF*np.log(ConcO_Cl/ConcI_Cl))+JKCC2)
    d_X = -dt/Ar*z_X*(g_X*np.log(ConcO_X/ConcI_X)

    ConcI_Na += d_Na   
    ConcI_K += d_K
    ConcI_Cl += d_Cl      
     
    #Osmolarity and volume adjustments
    OsmolI = ConcI_Na+ConcI_K+ConcI_Cl+ConcI_X
    w2 = w*(OsmolI/OsmolO)
    
    #Adjusting Concentrations based on new volumes
    ConcI_Na *= w/w2
    ConcI_K *= w/w2
    ConcI_Cl *= w/w2
    ConcI_X *= w/w2
    
    #Updating Arrays and counters
    w = w2
    
    if t >= ctr*ts :
        Vm_Arr.append(Vm*1e3)
        ECl_Arr.append(ECl*1e3)
        EK_Arr.append(EK*1e3)
        K_Arr.append(ConcI_K*1e3)
        Na_Arr.append(ConcI_Na*1e3)
        Cl_Arr.append(ConcI_Cl*1e3)
        X_Arr.append(ConcI_X*1e3)
        w_Arr.append(100*(1e5)*((3/(4*np.pi))*w)**(1/3))
        t_Arr.append(t)
        ctr += 1
    
        
    t = t+dt      
print('Simulation complete')
print('Final chloride concentration:', Cl_Arr[-1], ' M')
print('Final cell volume:', w_Arr[-1], 'L')

SyntaxError: ignored

ERROR! Session/line number was not unique in database. History logging moved to new session 63


## Plotting results

In [0]:
plt.plot(t_Arr,Na_Arr,label ="Na")
plt.plot(t_Arr,Cl_Arr,label="Cl")
plt.plot(t_Arr,K_Arr,label="K")
plt.plot(t_Arr,X_Arr,label="X")

SwitchOffX_Arr =[]
SwitchOnX_Arr= []
SwitchOnY_Arr=[]
SwitchOffY_Arr=[]
for a in range(0,150):
  SwitchOffX_Arr.append(t_off)
  SwitchOnX_Arr.append(t_on)
  SwitchOnY_Arr.append(a)
  SwitchOffY_Arr.append(a)

plt.plot(SwitchOnX_Arr,SwitchOnY_Arr,'k:')
plt.plot(SwitchOffX_Arr,SwitchOffY_Arr,'k:')
plt.title("Ionic concentration changes")
plt.xlabel("Time (s)")
plt.ylabel("Intracellular ionic concentrations (M)")
plt.legend(loc = 'upper right')
sns.despine()
plt.annotate('ATPase On', xy=(t_on, 150))
plt.annotate('ATPase Off', xy=(t_off, 150))


Visualizing the membrane potential changes with time

In [0]:
plt.plot(t_Arr,EK_Arr,label ="EK")
plt.plot(t_Arr,ECl_Arr,label="ECl")
plt.plot(t_Arr,Vm_Arr,label="Vm")

SwitchOffX_Arr =[]
SwitchOnX_Arr= []
SwitchOnY_Arr=[]
SwitchOffY_Arr=[]
for a in range(-120,150):
  SwitchOffX_Arr.append(t_off)
  SwitchOnX_Arr.append(t_on)
  SwitchOnY_Arr.append(a)
  SwitchOffY_Arr.append(a)

plt.plot(SwitchOnX_Arr,SwitchOnY_Arr,'k:')
plt.plot(SwitchOffX_Arr,SwitchOffY_Arr,'k:')
plt.title("Changes in cell potentials")
plt.xlabel("Time (s)")
plt.ylabel("Voltage (V)")
plt.legend(loc = 'upper right')
sns.despine()
plt.annotate('ATPase On', xy=(t_on, 150))
plt.annotate('ATPase Off', xy=(t_off, 150))



Visualizing cell volume changes with time

In [0]:
plt.plot(t_Arr,w_Arr,label="Volume (pL)")


SwitchOffX_Arr =[]
SwitchOnX_Arr= []
SwitchOnY_Arr=[]
SwitchOffY_Arr=[]
for a in range(300,530):
  SwitchOffX_Arr.append(t_off)
  SwitchOnX_Arr.append(t_on)
  SwitchOnY_Arr.append(a)
  SwitchOffY_Arr.append(a)
  
plt.plot(SwitchOnX_Arr,SwitchOnY_Arr,'k:')
plt.plot(SwitchOffX_Arr,SwitchOffY_Arr,'k:')
sns.despine()

plt.annotate('ATPase On', xy=(t_on, max(w_Arr)))
plt.annotate('ATPase Off', xy=(t_off, max(w_Arr)))
